In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.set_printoptions(linewidth=200)
torch.cuda.is_available()

True

In [2]:
from src.vlstm_fwbw_v0.interface import vlstm_fwbw_torch_obw, vlstm_fwbw_cuda

from src.vlstm_fwbw_v0.interface import vlstm_fw_torch, vlstm_fw_cuda
from src.vlstm_fwbw_v0.interface import vlstm_bw_torch_obw, vlstm_bw_cuda
from src.vlstm_fwbw_v0.torch_impl import vlstm_fw_tiled_torch

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt220cu121/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt220cu121/lib


Using /home/max/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu121/vlstm_fwbw_v0/build.ninja...
Building extension module vlstm_fwbw_v0...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /home/max/miniconda3/envs/xlstmpt220cu121/bin/nvcc --generate-dependencies-with-compile --dependency-output kernel_bw.cuda.o.d -ccbin /home/max/miniconda3/envs/xlstmpt220cu121/bin/x86_64-conda-linux-gnu-cc -DTORCH_EXTENSION_NAME=vlstm_fwbw_v0 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC -isystem /home/max/miniconda3/envs/xlstmpt220cu121/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERS

Loading extension module vlstm_fwbw_v0...


## CUDA vLSTM backward

In [3]:
S = 32 #16 #8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 8 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [4]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1) / 10.
# igs = torch.zeros((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
igs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
fgs = torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
fgs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE)

dHs = qs.clone()
qs, qs.shape, len(qs.view(-1))

(tensor([[[[ 0.0000,  0.1000,  0.2000,  0.3000,  0.4000,  0.5000,  0.6000,  0.7000],
           [ 0.8000,  0.9000,  1.0000,  1.1000,  1.2000,  1.3000,  1.4000,  1.5000],
           [ 1.6000,  1.7000,  1.8000,  1.9000,  2.0000,  2.1000,  2.2000,  2.3000],
           [ 2.4000,  2.5000,  2.6000,  2.7000,  2.8000,  2.9000,  3.0000,  3.1000],
           [ 3.2000,  3.3000,  3.4000,  3.5000,  3.6000,  3.7000,  3.8000,  3.9000],
           [ 4.0000,  4.1000,  4.2000,  4.3000,  4.4000,  4.5000,  4.6000,  4.7000],
           [ 4.8000,  4.9000,  5.0000,  5.1000,  5.2000,  5.3000,  5.4000,  5.5000],
           [ 5.6000,  5.7000,  5.8000,  5.9000,  6.0000,  6.1000,  6.2000,  6.3000],
           [ 6.4000,  6.5000,  6.6000,  6.7000,  6.8000,  6.9000,  7.0000,  7.1000],
           [ 7.2000,  7.3000,  7.4000,  7.5000,  7.6000,  7.7000,  7.8000,  7.9000],
           [ 8.0000,  8.1000,  8.2000,  8.3000,  8.4000,  8.5000,  8.6000,  8.7000],
           [ 8.8000,  8.9000,  9.0000,  9.1000,  9.2000,  9.3000,

### bw kernel match direct call

In [5]:
hs_pt, n_pt, m_pt, _, matD_pt = vlstm_fw_torch(queries=qs, keys=ks, values=vs, igate_preact=igs, fgate_preact=fgs)

In [6]:
dQs_pt, dKs_pt, dVs_pt, dIgs_pt, dFgs_pt, delta_D_pt, delta_Dtilde_pt, delta_fbar_pt, mat_P_pt, mat_C_pt = vlstm_bw_torch_obw(
    delta_Htilde=dHs,
    queries=qs,
    keys=ks,
    values=vs,
    igate_preact=igs,
    fgate_preact=fgs,
    var_n=n_pt,
    var_m=m_pt,
)

In [7]:
def rev_cumsum(x, dim=-1):
    return x.flip(dims=(dim,)).cumsum(dim).flip(dims=(dim,))

In [18]:
# calculate fgrads trick
df = (dQs_pt * qs - dKs_pt * ks).sum(-1, keepdim=True)
df = rev_cumsum(df, dim=-2)
df_pt = df * torch.nn.functional.sigmoid(-fgs)
# df_pt

In [19]:
# df_pt - dFgs_pt

In [10]:
hs_cu, n_cu, m_cu, _ = vlstm_fw_cuda(mat_Q=qs, mat_K=ks, mat_V=vs, igate_preact=igs, fgate_preact=fgs)

before kernel dispatch - float32!
B: 1, NH: 1, S: 32, DH: 8
blocksxy: 1-2, threadsxy: 4-4, shared_mem in bytes: 5700
In FW-Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In FW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4


In [11]:
dQs_cu, dKs_cu, dVs_cu, dIgs_cu, dFgs_cu, matC_cu, deltaDcsChunkArr_cu, deltaDcsVec_cu = vlstm_bw_cuda(
    delta_Htilde=dHs,
    mat_Q=qs,
    mat_K=ks,
    mat_V=vs,
    igate_preact=igs,
    fgate_preact=fgs,
    n=n_pt,
    m=m_pt,
)

before kernel dispatch - float32!
B: 1, NH: 1, S: 32, DH: 8
blocksxy: 1-2, threadsxy: 4-4, shared_mem in bytes: 7648
In BW-Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In BW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4
blockIdx(x,y)=(0,1), FTIdx=0, ijIdx(i,j)=(1,1), sTileXYIdx(x,y)=(8,8), csDYdimGBIdx=8, dcs_cv=0.000000, reachedMD=1
blockIdx(x,y)=(0,1), FTIdx=1, ijIdx(i,j)=(1,1), sTileXYIdx(x,y)=(8,8), csDYdimGBIdx=9, dcs_cv=0.104561, reachedMD=1
blockIdx(x,y)=(0,1), FTIdx=2, ijIdx(i,j)=(1,1), sTileXYIdx(x,y)=(8,8), csDYdimGBIdx=10, dcs_cv=0.063287, reachedMD=1
blockIdx(x,y)=(0,1), FTIdx=3, ijIdx(i,j)=(1,1), sTileXYIdx(x,y)=(8,8), csDYdimGBIdx=11, dcs_cv=0.017909, reachedMD=1
blockIdx(x,y)=(0,1), FTIdx=4, ijIdx(i,j)=(1,1), sTileXYIdx(x,y)=(8,8), csDYdimGBIdx=12, dcs_cv=0.651773, reachedMD=1
blockIdx(x,y)=(0,1), FTIdx=5, ijIdx(i,j)=(1,1), sTileXYIdx(x,y)=(8,8), csDYdimGBIdx=13, dcs_cv=0.726957, reachedMD=1
blockIdx(x,y)=(0,1), FTIdx=6, ijIdx(i,j)=(1,1), sTileXYIdx(x,y

In [33]:
RTOL = 1e-5
ATOL = 1e-5
print(f"delta Q match: {torch.allclose(dQs_cu, dQs_pt, rtol=RTOL, atol=ATOL)}")
print(f"delta K match: {torch.allclose(dKs_cu, dKs_pt, rtol=RTOL, atol=ATOL)}")
print(f"delta V match: {torch.allclose(dVs_cu, dVs_pt, rtol=RTOL, atol=ATOL)}")
print(f"delta Igate match: {torch.allclose(dIgs_cu, dIgs_pt.squeeze(), rtol=RTOL, atol=ATOL)}")
print(f"delta Fgate match: {torch.allclose(dFgs_cu, delta_fbar_pt.squeeze(-1), rtol=RTOL, atol=ATOL)}")

delta Q match: True
delta K match: False
delta V match: False
delta Igate match: True
delta Fgate match: True


In [12]:
delta_Dtilde_pt.cumsum(-1).tril(-1)[:, :, :, 5:17]

tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [6.2363e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.000

In [13]:
deltaDcsVec_cu.unsqueeze(-1), deltaDcsVec_cu.shape

(tensor([[[[0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [0.0000e+00],
           [7.7359e-01],
           [3.7578e-01],
           [1.2229e-01],
           [8.2258e-02],
           [7.6685e-02],
           [7.1804e-02],
           [4.7018e-02],
           [2.3921e-02],
           [1.1210e-02],
           [3.8283e-03],
           [6.5380e-04],
           [4.0062e-04],
           [3.2340e-04],
           [2.8763e-04],
           [1.2127e-04],
           [1.0430e-04]]]], device='cuda:0'),
 torch.Size([1, 1, 32]))

In [14]:
dFgs_cu - delta_fbar_pt.squeeze(-1)

tensor([[[ 0.0000e+00,  2.7940e-09,  1.8626e-08,  1.6764e-08,  7.4506e-08,  7.4506e-08,  4.4703e-08,  5.9605e-08,  8.9407e-08,  1.0431e-07,  1.3411e-07,  1.7136e-07,  3.8147e-06,  2.1458e-06,
           7.1526e-07,  1.3113e-06,  2.2650e-06,  1.4305e-06,  9.5367e-07,  2.6226e-06,  3.0994e-06,  2.1458e-06,  4.7684e-07, -5.9605e-07,  5.9605e-08,  7.7486e-07,  9.5367e-07,  2.6226e-06,
           4.7684e-07,  0.0000e+00,  0.0000e+00,  5.9605e-07]]], device='cuda:0')

In [15]:
matC_cu[:, :, :, -10:] - delta_Dtilde_pt.cumsum(-1).tril(-1)[:, :, :, -10:]

tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,

In [35]:
matC_cu[:, :, :, -9:-8]

tensor([[[[0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0830],
          [0.0734],
          [0.0239],
          [0.0046],
          [0.0041],
          [0.0037],
          [0.0013],
          [0.0006]]]], device='cuda:0')

In [36]:
deltaDcsChunkArr_cu

tensor([[[[0.0830, 0.0734, 0.0239, 0.0046, 0.0041, 0.0037, 0.0013, 0.0006],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]]], device='cuda:0')

In [37]:
deltaDcsVec_cu, deltaDcsVec_cu.shape

(tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]], device='cuda:0'),
 torch.Size([1, 1, 16]))

In [34]:
delta_Dtilde_pt.cumsum(-1).tril(-1)[:, :, :, :-1]

tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [4.3366e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [8.9254e-04, 4.9177e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [5.7532e-04, 3.1699e-03, 3.5641e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [7.6602e-04, 4.2206e-03, 4.7456e-03, 3.1090e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [8.1001e-04, 4.4630e-03, 5.018

In [27]:
dQs_cu - dQs_pt

tensor([[[[ 1.0914e-11,  1.0914e-11,  1.0914e-11,  1.0914e-11,  1.0914e-11,  1.0914e-11,  1.0914e-11,  1.0914e-11],
          [ 1.4552e-10,  1.4552e-10,  1.4552e-10,  1.4552e-10,  1.4552e-10,  1.4552e-10,  1.4552e-10,  1.4552e-10],
          [ 1.4552e-10,  1.4552e-10,  1.4552e-10,  1.4552e-10,  1.4552e-10,  1.4552e-10,  1.4552e-10,  1.4552e-10],
          [ 1.1642e-09,  1.1642e-09,  1.1642e-09,  1.1642e-09,  1.1642e-09,  1.1642e-09,  1.1642e-09,  1.1642e-09],
          [ 1.3970e-09,  1.3970e-09,  1.3970e-09,  1.3970e-09,  1.3970e-09,  1.3970e-09,  1.3970e-09,  1.3970e-09],
          [ 1.1642e-09,  1.1642e-09,  1.1642e-09,  1.1642e-09,  1.1642e-09,  1.1642e-09,  1.1642e-09,  1.1642e-09],
          [ 1.3970e-09,  1.3970e-09,  1.3970e-09,  1.3970e-09,  1.3970e-09,  1.3970e-09,  1.3970e-09,  1.3970e-09],
          [ 1.1642e-09,  1.1642e-09,  1.1642e-09,  1.1642e-09,  1.1642e-09,  1.1642e-09,  1.1642e-09,  1.1642e-09],
          [ 6.9849e-10,  6.9849e-10,  6.9849e-10,  6.9849e-10,  6.9849e-

In [28]:
dKs_cu - dKs_pt

tensor([[[[2.9802e-08, 3.3528e-08, 2.9802e-08, 3.3528e-08, 3.3528e-08, 3.7253e-08, 2.9802e-08, 2.9802e-08],
          [1.4901e-07, 1.4901e-07, 2.0862e-07, 1.7881e-07, 2.0862e-07, 1.7881e-07, 2.0862e-07, 2.3842e-07],
          [2.2352e-08, 2.6077e-08, 2.2352e-08, 2.6077e-08, 2.6077e-08, 2.9802e-08, 2.9802e-08, 3.3528e-08],
          [8.3447e-07, 9.5367e-07, 8.3447e-07, 9.5367e-07, 9.5367e-07, 8.3447e-07, 1.0729e-06, 9.5367e-07],
          [7.1526e-07, 7.1526e-07, 5.9605e-07, 7.1526e-07, 7.1526e-07, 8.3447e-07, 7.1526e-07, 5.9605e-07],
          [5.3644e-07, 4.1723e-07, 3.5763e-07, 4.1723e-07, 4.7684e-07, 5.3644e-07, 4.1723e-07, 5.3644e-07],
          [2.9802e-07, 3.5763e-07, 2.9802e-07, 2.9802e-07, 2.9802e-07, 2.3842e-07, 4.1723e-07, 2.3842e-07],
          [9.5367e-07, 1.4305e-06, 1.1921e-06, 7.1526e-07, 1.1921e-06, 1.1921e-06, 1.1921e-06, 1.1921e-06],
          [5.9605e-07, 5.9605e-07, 8.3447e-07, 7.1526e-07, 5.9605e-07, 7.1526e-07, 5.9605e-07, 4.7684e-07],
          [1.4305e-06, 1.668

In [29]:
dVs_cu - dVs_pt

tensor([[[[2.6077e-08, 3.3528e-08, 3.7253e-08, 2.9802e-08, 2.9802e-08, 4.4703e-08, 3.7253e-08, 2.9802e-08],
          [1.7881e-07, 1.7881e-07, 2.3842e-07, 2.0862e-07, 2.0862e-07, 2.0862e-07, 1.7881e-07, 2.0862e-07],
          [2.2352e-08, 2.2352e-08, 2.2352e-08, 2.6077e-08, 2.6077e-08, 2.6077e-08, 3.3528e-08, 2.9802e-08],
          [9.5367e-07, 9.5367e-07, 9.5367e-07, 1.0729e-06, 1.0729e-06, 1.1921e-06, 9.5367e-07, 1.0729e-06],
          [7.1526e-07, 7.1526e-07, 7.1526e-07, 8.3447e-07, 7.1526e-07, 7.1526e-07, 7.1526e-07, 5.9605e-07],
          [5.3644e-07, 4.7684e-07, 4.1723e-07, 4.7684e-07, 4.7684e-07, 4.7684e-07, 4.7684e-07, 4.1723e-07],
          [3.5763e-07, 2.9802e-07, 2.3842e-07, 3.5763e-07, 2.3842e-07, 2.9802e-07, 3.5763e-07, 4.1723e-07],
          [1.1921e-06, 1.4305e-06, 9.5367e-07, 7.1526e-07, 9.5367e-07, 1.1921e-06, 7.1526e-07, 9.5367e-07],
          [5.3644e-07, 5.9605e-07, 4.7684e-07, 4.7684e-07, 4.7684e-07, 4.7684e-07, 5.9605e-07, 3.5763e-07],
          [1.4305e-06, 1.430

In [30]:
dIgs_cu - dIgs_pt.squeeze(-1), dIgs_cu.shape

(tensor([[[ 2.3283e-09,  1.4901e-08,  1.8626e-09,  9.6858e-08,  5.9605e-08,  3.7253e-08,  2.2352e-08,  7.4506e-08,  3.7253e-08,  1.3411e-07,  5.2154e-08,  6.4373e-06,  7.1526e-07,  4.0233e-07,
            2.2650e-06,  2.0266e-06,  4.7684e-07,  1.4305e-06,  3.3379e-06,  1.9073e-06,  3.2783e-07,  5.9605e-08,  0.0000e+00,  8.9407e-08,  1.5497e-06,  1.3113e-06,  4.2915e-06, -4.7684e-07,
            1.1921e-07,  1.1921e-07,  1.6689e-06,  1.1921e-07]]], device='cuda:0'),
 torch.Size([1, 1, 32]))

In [31]:
dFgs_cu - delta_fbar_pt.squeeze(-1)

tensor([[[ 0.0000e+00,  2.7940e-09,  1.8626e-08,  1.6764e-08,  7.4506e-08,  7.4506e-08,  4.4703e-08,  5.9605e-08,  8.9407e-08,  1.0431e-07,  1.3411e-07,  1.7136e-07,  3.8147e-06,  2.1458e-06,
           7.1526e-07,  1.3113e-06,  2.2650e-06,  1.4305e-06,  9.5367e-07,  2.6226e-06,  3.0994e-06,  2.1458e-06,  4.7684e-07, -5.9605e-07,  5.9605e-08,  7.7486e-07,  9.5367e-07,  2.6226e-06,
           4.7684e-07,  0.0000e+00,  0.0000e+00,  5.9605e-07]]], device='cuda:0')

In [ ]:
# matC_cu - mat_P_pt
# matC_cu - mat_C_pt

In [ ]:
matC_cu[:, :, :, :-1]

tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [4.3366e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [8.9254e-04, 4.9177e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [5.7532e-04, 3.1699e-03, 3.5641e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [7.6602e-04, 4.2206e-03, 4.7456e-03, 3.1090e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [8.1001e-04, 4.4630e-03, 5.018

In [ ]:
delta_Dtilde_pt.cumsum(-1).tril(-1)[:, :, :, :-1]

tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [4.3366e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [8.9254e-04, 4.9177e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [5.7532e-04, 3.1699e-03, 3.5641e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [7.6602e-04, 4.2206e-03, 4.7456e-03, 3.1090e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [8.1001e-04, 4.4630e-03, 5.018

In [ ]:
(delta_Dtilde_pt.cumsum(-1).tril(-1) - matC_cu)[:,:,:,:-2]

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-2.0373e-10,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-2.3283e-10, -1.8626e-09,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-5.8208e-10, -3.9581e-09, -4.4238e-09,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-4.0745e-10, -2.7940e-09, -3.2596e-09, -2.4214e-08,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-2.9104e-10, -2.328

In [ ]:
dFgs_cu, delta_fbar_pt.squeeze(-1)

(tensor([[[0.0000, 0.0048, 0.0240, 0.0214, 0.1169, 0.1556, 0.1346, 0.1095, 0.1947, 0.0474, 0.0631, 0.0416, 1.5484, 1.1013, 0.4477, 0.4307]]], device='cuda:0'),
 tensor([[[0.0000, 0.0048, 0.0240, 0.0214, 0.1169, 0.1556, 0.1346, 0.1095, 0.1947, 0.1592, 0.1014, 0.0560, 1.5582, 1.1070, 0.4496, 0.4314]]], device='cuda:0'))

In [15]:
dFgs_cu - delta_fbar_pt.squeeze(-1)

tensor([[[ 0.0000e+00,  2.7940e-09,  1.8626e-08,  1.6764e-08,  7.4506e-08,  7.4506e-08,  4.4703e-08,  5.9605e-08,  8.9407e-08,  1.0431e-07,  1.3411e-07,  1.7136e-07,  3.8147e-06,  2.1458e-06,
           7.1526e-07,  1.3113e-06,  2.2650e-06,  1.4305e-06,  9.5367e-07,  2.6226e-06,  3.0994e-06,  2.1458e-06,  4.7684e-07, -5.9605e-07,  5.9605e-08,  7.7486e-07,  9.5367e-07,  2.6226e-06,
           4.7684e-07,  0.0000e+00,  0.0000e+00,  5.9605e-07]]], device='cuda:0')

In [ ]:
deltaDcsVec_cu, deltaDcsVec_cu.shape

(tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]], device='cuda:0'),
 torch.Size([1, 1, 16]))

### bw kernel match through autograd

In [ ]:
fgs_pt = fgs.clone().detach().requires_grad_(True)
igs_pt = igs.clone().detach().requires_grad_(True)
qs_pt = qs.clone().detach().requires_grad_(True)
ks_pt = ks.clone().detach().requires_grad_(True)
vs_pt = vs.clone().detach().requires_grad_(True)

In [ ]:
# pytorch version QKV product (To test if this is still correct after changes to the code.)
# at some point we have to compare to the vlstm_fw_torch version.
# rs = qs @ ks.transpose(-1, -2) @ vs
# rs, rs.shape

In [ ]:
# rs_torch = (qs @ ks.transpose(-1, -2) * torch.tril(torch.ones((B, NH, S, S))).to(device=DEVICE, dtype=DTYPE)) @ vs
# rs_torch, rs_torch.shape

In [ ]:
# pytorch version
hs_pt, n_pt, m_pt = vlstm_fwbw_torch_obw(
    queries=qs_pt,
    keys=ks_pt,
    values=vs_pt,
    igate_preact=igs_pt,
    fgate_preact=fgs_pt,
)
hs_pt, hs_pt.shape

(tensor([[[[3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05],
           [2.5971e-04, 2.5971e-04, 2.5971e-04, 2.5971e-04, 2.5971e-04, 2.5971e-04, 2.5971e-04, 2.5971e-04],
           [3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04],
           [1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03],
           [1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03],
           [2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03],
           [2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03],
           [1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03],
           [2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03],
           [3.2163e

In [ ]:
fgs_cu = fgs.clone().detach().requires_grad_(True)
igs_cu = igs.clone().detach().requires_grad_(True)
qs_cu = qs.clone().detach().requires_grad_(True)
ks_cu = ks.clone().detach().requires_grad_(True)
vs_cu = vs.clone().detach().requires_grad_(True)

In [ ]:
# cuda kernel
hs_cu, n_cu, m_cu, matD_cu = vlstm_fwbw_cuda(mat_Q=qs_cu, mat_K=ks_cu, mat_V=vs_cu, igate_preact=igs_cu.squeeze(-1), fgate_preact=fgs_cu.squeeze(-1))
hs_cu, hs_cu.shape

before kernel dispatch - float32!
B: 1, NH: 1, S: 16, DH: 8
blocksxy: 1-2, threadsxy: 4-4, shared_mem in bytes: 5700
In FW-Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In FW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4


(tensor([[[[3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05, 3.9268e-05],
           [2.5972e-04, 2.5972e-04, 2.5972e-04, 2.5972e-04, 2.5972e-04, 2.5972e-04, 2.5972e-04, 2.5972e-04],
           [3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04, 3.5444e-04],
           [1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03, 1.0614e-03],
           [1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03, 1.9845e-03],
           [2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03, 2.4016e-03],
           [2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03, 2.2950e-03],
           [1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03, 1.9563e-03],
           [2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03, 2.1884e-03],
           [3.2163e

In [ ]:
# forward match
hs_pt - hs_cu

tensor([[[[-1.4552e-11, -1.4552e-11, -1.4552e-11, -1.4552e-11, -1.4552e-11, -1.4552e-11, -1.4552e-11, -1.4552e-11],
          [-1.4552e-10, -1.4552e-10, -1.4552e-10, -1.4552e-10, -1.4552e-10, -1.4552e-10, -1.4552e-10, -1.4552e-10],
          [-1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10],
          [-1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09],
          [-1.6298e-09, -1.6298e-09, -1.6298e-09, -1.6298e-09, -1.6298e-09, -1.6298e-09, -1.6298e-09, -1.6298e-09],
          [-6.9849e-10, -6.9849e-10, -6.9849e-10, -6.9849e-10, -6.9849e-10, -6.9849e-10, -6.9849e-10, -6.9849e-10],
          [-1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09],
          [-1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09, -1.1642e-09],
          [-2.3283e-10, -2.3283e-10, -2.3283e-10, -2.3283e-10, -2.3283e-

In [ ]:
hs_pt.sum().backward()

ValueError: too many values to unpack (expected 5)

In [ ]:
qs_pt.grad, ks_pt.grad, vs_pt.grad, igs_pt.grad, fgs_pt.grad

(tensor([[[[0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
           [0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002],
           [0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002],
           [0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004],
           [0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006],
           [0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006],
           [0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004],
           [0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003],
           [0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003],
           [0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004],
           [0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002],
           [0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011],
           [0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010],

In [ ]:
hs_cu.sum().backward()

before kernel dispatch - float32!
B: 1, NH: 1, S: 16, DH: 8
blocksxy: 1-2, threadsxy: 4-4, shared_mem in bytes: 7136
In BW-Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In BW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4


In [ ]:
qs_cu.grad, ks_cu.grad, vs_cu.grad, igs_cu.grad, fgs_cu.grad

(tensor([[[[0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.]]]], device='cuda:0'),
 tensor([[[[0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.